In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Read in the data

df = pd.read_csv("C:\\Users\\joaju\\OneDrive\\Projects\\Mushroom_Classifier\\mushroom_cleaned.csv")

In [3]:
X = pd.get_dummies(df.drop(['class'], axis = 1))
y = df['class']

In [4]:
# Outputs of the train_test_split function, so that training from X and y the result is stored in the four variables.

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [5]:
X_train.head()

,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season
5047,676,2,1,11,0.079154,1064,6,1.804273
1002,872,2,0,10,1.571275,1648,11,0.888450
41596,508,2,6,5,0.504906,395,11,0.943195
50425,443,3,0,10,2.013794,0,2,0.888450
26260,1425,6,1,5,0.218419,3011,11,0.888450


In [6]:
y_train.head()

5047     1
1002     0
41596    0
50425    1
26260    1
Name: class, dtype: int64

In [7]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

Build our Model by adding layers

In [8]:
model = Sequential()

# Add layers to the neural networks
# units : how many neurons in the dense layer
# activation function
model.add(Dense(units=32, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

c:\Users\joaju\anaconda4\envs\newenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Compile our Model

In [9]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

I have detected that our model does not increase in accuracy from 200 to 500 epochs.
So we will apply early stopping.

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

In [16]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Retrain the model applying the early stopping
history = model.fit(X_train, y_train, epochs=200, 
                    batch_size=64, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/200
541/541 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5499 - loss: 0.6711 - val_accuracy: 0.5441 - val_loss: 0.6750
Epoch 2/200
541/541 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5433 - loss: 0.6764 - val_accuracy: 0.5459 - val_loss: 0.6753
Epoch 3/200
541/541 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5512 - loss: 0.6748 - val_accuracy: 0.5459 - val_loss: 0.6741
Epoch 4/200
541/541 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5495 - loss: 0.6747 - val_accuracy: 0.5459 - val_loss: 0.6720
Epoch 5/200
541/541 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5519 - loss: 0.6722 - val_accuracy: 0.5459 - val_loss: 0.6712
Epoch 6/200
541/541 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5495 - loss: 0.6712 - val_accuracy: 0.5459 - val_loss: 0.6707
Epoch 7/200
541/541 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5501 - loss: 0.6714 - val_accuracy: 0.5459 - val_loss: 0.6709
Epoch 8/200
541/541 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5478 - loss: 0.6718 - val_accu

Re-evaluate the Early Stopping Model

In [41]:
loss, accuracy = model.evaluate(X_test, y_test)

338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5539 - loss: 0.6689


Save and Load our Model

In [42]:
# Saving our model in a file or directory so that it saves
# the training configuration such as optimizer, loss and metrics.
# it can be loaded and used later without having to retrain it.

model.save('tfmodel.keras')

In [43]:
# Delete the model from the memory in this environment.

del model

In [44]:
# Later on load_model function will load our saved model as follows.

model = load_model('tfmodel.keras')